# Google Spanner

> [Spanner](https://cloud.google.com/spanner) is a highly scalable database that combines unlimited scalability with relational semantics, such as secondary indexes, strong consistency, schemas, and SQL providing 99.999% availability in one easy solution.

This notebook goes over how to use `Spanner` for GraphRAG with `SpannerGraphStore` class.

Learn more about the package on [GitHub](https://github.com/googleapis/langchain-google-spanner-python/).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-spanner-python/blob/main/docs/graph_store.ipynb)

## Before You Begin

To run this notebook, you will need to do the following:

 * [Create a Google Cloud Project](https://developers.google.com/workspace/guides/create-project)
 * [Enable the Cloud Spanner API](https://console.cloud.google.com/flows/enableapi?apiid=spanner.googleapis.com)
 * [Create a Spanner instance](https://cloud.google.com/spanner/docs/create-manage-instances)
 * [Create a Spanner database](https://cloud.google.com/spanner/docs/create-manage-databases)

### 🦜🔗 Library Installation
The integration lives in its own `langchain-google-spanner` package, so we need to install it.

In [ ]:
%pip install --upgrade --quiet langchain-google-spanner langchain-google-vertexai langchain-experimental json-repair wikipedia pyvis youtube-transcript-api

**Colab only:** Uncomment the following cell to restart the kernel or use the button to restart the kernel. For Vertex AI Workbench you can restart the terminal using the button on top.

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

### 🔐 Authentication
Authenticate to Google Cloud as the IAM user logged into this notebook in order to access your Google Cloud Project.

* If you are using Colab to run this notebook, use the cell below and continue.
* If you are using Vertex AI Workbench, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [ ]:
from google.colab import auth

auth.authenticate_user()

### ☁ Set Your Google Cloud Project
Set your Google Cloud project so that you can leverage Google Cloud resources within this notebook.

If you don't know your project ID, try the following:

* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113).

In [ ]:
# @markdown Please fill in the value below with your Google Cloud project ID and then run the cell.

PROJECT_ID = ""  # @param {type:"string"}

# Set the project id
!gcloud config set project {PROJECT_ID}
%env GOOGLE_CLOUD_PROJECT={PROJECT_ID}

### 💡 API Enablement
The `langchain-google-spanner` package requires that you [enable the Spanner API](https://console.cloud.google.com/flows/enableapi?apiid=spanner.googleapis.com) in your Google Cloud Project.

In [ ]:
# enable Spanner API
!gcloud services enable spanner.googleapis.com

## Basic Usage

### Set Spanner database values
Find your database values, in the [Spanner Instances page](https://console.cloud.google.com/spanner?_ga=2.223735448.2062268965.1707700487-2088871159.1707257687).

In [ ]:
# @title Set Your Values Here { display-mode: "form" }
INSTANCE = ""  # @param {type: "string"}
DATABASE = ""  # @param {type: "string"}
GRAPH_NAME = ""  # @param {type: "string"}

### SpannerGraphStore

To initialize the `SpannerGraphStore` class you need to provide 3 required arguments and other arguments are optional and only need to pass if it's different from default ones

1.   a Spanner instance id;
2.   a Spanner database id belongs to the above instance id;
3.   a Spanner graph name used to create a graph in the above database.

In [ ]:
from langchain_google_spanner import SpannerGraphStore

graph_store = SpannerGraphStore(
    instance_id=INSTANCE,
    database_id=DATABASE,
    graph_name=GRAPH_NAME,
)

#### Add Graph Documents
To add graph documents in the graph store.

In [ ]:
# @title Get graph documents from Wikipedia & YouTube
from langchain_community.document_loaders import WikipediaLoader, YoutubeLoader
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_google_vertexai.llms import VertexAI

llm = VertexAI(model_name="text-bison")
llm_transformer = LLMGraphTransformer(llm=llm)


def get_document_from_wiki(query):
    print("Fetching document from Wikipedia: `%s`" % query)
    loader = WikipediaLoader(query=query, load_max_docs=1)
    return loader.load()


def get_document_from_youtube(link):
    print("Fetching document from YouTube: `%s`" % link)
    loader = YoutubeLoader.from_youtube_url(link)
    return loader.load()


document_batch = list(map(get_document_from_wiki, ["Google"]))
document_batch += list(
    map(get_document_from_youtube, ["https://www.youtube.com/watch?v=WsEQjeZoEng"])
)
graph_documents = llm_transformer.convert_to_graph_documents(
    [document for documents in document_batch for document in documents]
)

Fetching document from Wikipedia: `Google`
Fetching document from YouTube: `https://www.youtube.com/watch?v=WsEQjeZoEng`


In [ ]:
for doc in graph_documents:
    print(doc.source.page_content[:100])
    print(doc.nodes)
    print(doc.relationships)
    print()

Google LLC (  GOO-ghəl) is an American multinational corporation and technology company focusing on 
[Node(id='Google LLC', type='Company', properties={}), Node(id='Larry Page', type='Person', properties={}), Node(id='Sergey Brin', type='Person', properties={})]
[Relationship(source=Node(id='Google LLC', type='Company', properties={}), target=Node(id='Larry Page', type='Person', properties={}), type='FOUNDED_BY', properties={}), Relationship(source=Node(id='Google LLC', type='Company', properties={}), target=Node(id='Sergey Brin', type='Person', properties={}), type='FOUNDED_BY', properties={})]

[MUSIC PLAYING] [APPLAUSE] SUNDAR PICHAI: At Google, we
are fully in our Gemini era. Today, all of o
[Node(id='Google', type='Company', properties={}), Node(id='Gemini', type='Product', properties={}), Node(id='summarize emails', type='Feature', properties={})]
[Relationship(source=Node(id='Google', type='Company', properties={}), target=Node(id='Gemini', type='Product', properties={}), type='

In [ ]:
graph_store.cleanup()

for graph_document in graph_documents:
    graph_store.add_graph_documents([graph_document])

Waiting for DDL operations to complete...
Waiting for DDL operations to complete...
Waiting for DDL operations to complete...
Waiting for DDL operations to complete...
Insert nodes of type `Company`...
Executed DML statement: 1 rows updated
Insert nodes of type `Person`...
Executed DML statement: 2 rows updated
Insert edges of type `FOUNDED_BY`...
Executed DML statement: 2 rows updated
Waiting for DDL operations to complete...
Insert nodes of type `Company`...
Executed DML statement: 1 rows updated
Insert nodes of type `Product`...
Executed DML statement: 1 rows updated
Insert nodes of type `Feature`...
Executed DML statement: 1 rows updated
Insert edges of type `HAS_PRODUCT`...
Executed DML statement: 1 rows updated
Insert edges of type `HAS_FEATURE`...
Executed DML statement: 1 rows updated


#### Query the graph
To traverse the graph in the graph store.

In [ ]:
sample_query = """
  GRAPH {}
  MATCH (n:Company WHERE REGEXP_CONTAINS(n.id, 'Google')) -[e]-{{1, 2}} (m)
  RETURN ARRAY_AGG(DISTINCT m.id) AS google_related_nodes
""".format(
    GRAPH_NAME
)

print(graph_store.query(sample_query))

[{'google_related_nodes': ['Larry Page', 'Gemini', 'Google', 'summarize emails', 'Sergey Brin', 'Google LLC']}]


#### Visualize the graph

In [ ]:
from pyvis.network import Network
from IPython.core.display import display, HTML

net = Network(
    notebook=True,
    cdn_resources="remote",
    bgcolor="#222222",
    font_color="white",
    height="500px",
    width="50%",
    directed=True,
)

node_query = """
  GRAPH {}
  MATCH (n)
  RETURN n.id
""".format(
    GRAPH_NAME
)

edge_query = """
  GRAPH {}
  MATCH -[e]->
  RETURN e.id AS src_id, e.target_id AS dst_id, labels(e)[0] AS label
""".format(
    GRAPH_NAME
)

nodes = graph_store.query(node_query)
edges = graph_store.query(edge_query)

net.add_nodes([node["id"] for node in nodes])
for edge in edges:
    net.add_edge(edge["src_id"], edge["dst_id"], title=edge["label"])

net.toggle_physics(True)
net.show("graph.html")
display(HTML("graph.html"))

graph.html


#### Clean up the graph

> USE IT WITH CAUTION!

Clean up all the nodes/edges in your graph and remove your graph definition.

In [ ]:
graph_store.cleanup()

Waiting for DDL operations to complete...
Waiting for DDL operations to complete...
Waiting for DDL operations to complete...
